In [27]:
import pandas as pd
import os, random, glob, json
from sentence_transformers import SentenceTransformer
import nltk
import numpy as np
from sklearn.svm import LinearSVC

In [28]:

files = glob.glob('/Users/adamtucker/Downloads/tab_feature_files_copy/*.csv')
empty_list = []
final_list = []
header_names = ["sentence_id", "token_id", "token", "lemma", "pos", "term_id", "pol/mod", "poldomain", "aspect_training", "entity", "property", "phrase_type", "sentiment" , ]

for file in files:
    #print(file)
    frame= pd.read_csv(file,sep="\t", header=None, names=header_names)
    list_of_tokens = []
    list_of_tokens.append(frame['token'].tolist())
    

    for a in list_of_tokens:
        #string_tokens = str(list_of_tokens)
        string_per_file = " ".join(a)
        #print(string_per_file)
        break
        

    list_of_sentiment = []
    list_of_sentiment.append(frame['sentiment'].tolist())
    #print(list_of_sentiment)

    score = 0
    for a in list_of_sentiment:
        for b in a:
            if b == "B-negative":
                score += -1
            if b == "B-positive":
                score += 1
    #print(score)
    sentiment = ""
    if score > 0:
        sentiment = "positive"
    elif score == 0:
        sentiment = "neutral"
    else:
        sentiment = "negative"
    #print(sentiment)
    sentiment_dict = dict()
    sentiment_dict[string_per_file] = sentiment
    empty_list.append(string_per_file)
    
    #print(sentiment_dict)
    final_list.append(sentiment_dict)
#print(final_list)    
    #turn into dataframe



In [29]:
final_list[0]
text_list = []
label_list = []
for d in final_list:
    for key, value in d.items():
        text_list.append(key)
        label_list.append(value)
        

df['text'] = text_list
df['label'] = label_list
print(len(text_list))
print (len(label_list))




506
506


In [30]:
data_text={'text':text_list, 'label': label_list}
df2=pd.DataFrame(data=data_text)
df2

,text,label
0,in perfect agreement with the late poll on the...,negative
1,"LONDON , Jan @card@ ( AFP ) -- British Foreign...",negative
2,.start Benjamin Franklin Federal Savings & Loa...,neutral
3,"Kinshasa , July @card@ ( AFP ) - some @card@ r...",negative
4,"June @card@ , @card@ Name Address City , St Zi...",positive
...,...,...
501,"follow the presidential vote in Zimbabwe , the...",negative
502,Six day that shake Venezuela Caracas ( AFP ) -...,negative
503,"Osaka , Dec. 7 Kyodo -- a Japanese nongovernme...",positive
504,"Taipei , Oct. 8 ( CNA ) -- the United States m...",positive


In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
texts = df2.text.to_list()

embedded_texts = []
for text in texts:
    sentences = nltk.sent_tokenize(text)
    if len(sentences) >=6:
        sentence_embeddings = model.encode(sentences[:6])
    else:
        for n in range (6-len(sentences)):
            sentences.append(sentences[0])
    embedded_texts.append(sentence_embeddings)

In [26]:

for text in texts:
    sentences = nltk.sent_tokenize(text)
    if len(sentences) >=6:
        sentence_embeddings = model.encode(sentences[:6])
    else:
        for n in range (6-len(sentences)):
            sentences.append(sentences[0])
        sentence_embeddings = model.encode(sentences[:6])
    embedded_texts.append(sentence_embeddings)

KeyboardInterrupt: 

In [ ]:
# use concatenated list or mean list in the train_test_split function below

concatenated_list = []
mean_list=[]

for six_embeds in embedded_texts:
    new = np.concatenate(six_embeds)
    concatenated_list.append(new)
    mean_embeds_per_text = np.mean(six_embeds, axis = 0)
    mean_list.append(mean_embeds_per_text)

In [ ]:
training_labels = list(df2.label)

In [14]:
from sklearn.model_selection import train_test_split
# mean_list can be exchanged for concatenated_list
X_train, X_test, y_train, y_test = train_test_split(mean_list, training_labels, test_size=0.33, random_state=42)

NameError: name 'mean_list' is not defined

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

